In [84]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [125]:
# CSV 파일 불러오기
df = pd.read_csv('./merged_rag_data.csv')
new_df1 = pd.read_csv('1_youtube_puppy_dictionary.csv') 
new_df2 = pd.read_csv('2_youtube_doghumanbond_data.csv') 
new_df3 = pd.read_csv('5_youtube_dogconseling_data.csv') 

In [126]:
import pandas as pd
import json

# 1. JSON 파일 불러오기
with open('./6_wayo_final_data (1).json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

# 2. JSON → DataFrame
new_df4 = pd.DataFrame(json_data)

# content 구성: 여러 필드를 이어 붙이기
new_df4['title'] = new_df4['질문 제목']
new_df4['content'] = (
    "Q. " + new_df4['증상과 행동'] + "\n" + new_df4['시작된 시점'] + "\n" + new_df4['보호자님 반응'] + "\n" +
    "A. " + new_df4['원인 분석'] + "\n" + new_df4['솔루션 제안']
)

# 필요한 컬럼만 추출 및 컬럼명 정리
new_df4 = new_df4[['title', 'content', 'url']]
new_df4.rename(columns={'url': 'URL'}, inplace=True)

# image 컬럼 추가 (없으면 빈 문자열 또는 None으로 처리)
new_df4['image'] = ''  # 또는 None

# content 줄바꿈 제거
new_df4['content'] = new_df4['content'].str.replace(r'[\r\n]+', ' ', regex=True).str.strip()


### 1. df 통합

In [127]:
# 데이터 병합 함수(병합할 df 두 개)
def merge_dataframes(df1, df2):

    # 컬럼명 소문자로 통일
    df1.columns = df1.columns.str.lower()
    df2.columns = df2.columns.str.lower()
    new_df = pd.concat([df1, df2], ignore_index=True)

    return new_df

In [128]:
df = merge_dataframes(df, new_df1)
df = merge_dataframes(df, new_df2)
df = merge_dataframes(df, new_df3)
df1 = merge_dataframes(df, new_df4)
len(df1)

4240

### 2. 줄바꿈 정리

In [129]:
def clean_newlines(df, col_name):
    df[col_name] = df[col_name].str.replace(r'[\r\n]+', ' ', regex=True)
    df[col_name] = df[col_name].str.replace(r'\s+', ' ', regex=True)
    return df

In [130]:
df2 = clean_newlines(df1,'content')

In [ ]:
# df2.head()

### 3. 결측치 제거

In [134]:
"""
결측값, url 형식 이상인 행 제거
content 결측값 있는 행 제거
url이 http 또는 https로 시작하지 않는 행 제거
"""

def remove_missing(df):
    before = len(df)
    df_clean = df.dropna(subset=['content'])
    df_clean = df_clean[df_clean['url'].str.startswith(('http://', 'https://'), na=False)]
    after = len(df_clean)
    print(f"결측치/잘못된 URL 제거 전: {before}, 제거 후: {after}")
    return df_clean.reset_index(drop=True)


In [159]:
df3 = remove_missing(df2)
len(df3), len(df2)

결측치/잘못된 URL 제거 전: 4240, 제거 후: 4238


(4238, 4240)

### 4. 불용어 제거

In [147]:
import re

def remove_stopwords(df, col_name, syllables=None):
    if syllables is None:
        syllables = ['오', '아', '으', '어', '이']

    # 패턴: 감탄사 단어가 3개 이상 연속
    pattern = r'(?:\b(?:' + '|'.join(syllables) + r')\b[\s]*){3,}'

    # 제거 전 미리보기
    df_noise = df[df[col_name].str.contains(pattern, regex=True, na=False)]
    print(f"=== '{col_name}' 컬럼에서 불용어(감탄사 3개 이상 반복) 포함 예시 ===")
    pd.set_option('display.max_colwidth', None)
    print(df_noise[[col_name]].head(5))

    # 실제 제거
    df[col_name] = df[col_name].str.replace(pattern, '', regex=True)

    return df

In [162]:
df4 = remove_stopwords(df3, 'content')

=== 'content' 컬럼에서 불용어(감탄사 3개 이상 반복) 포함 예시 ===
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [160]:
df3.iloc[1653]

title                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [163]:
df4.iloc[1653]

title                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

### 5. 중복 콘텐츠 제거

In [164]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

def remove_similar_entries(df, title_col='title', content_col='content', threshold=0.99, preview=True):
    """
    TF-IDF 기반 유사 항목 제거 함수
    - title과 content를 합쳐 유사도 계산
    - 유사도가 threshold 이상인 항목은 제거
    - 중복 후보 미리보기도 가능
    """
    df = df.reset_index(drop=True).copy()
    
    # 줄 바꿈 및 결측값 대응
    df[title_col] = df[title_col].fillna('').astype(str)
    df[content_col] = df[content_col].fillna('').astype(str)

    # title + content 합치기
    df['combined'] = df[title_col] + ' ' + df[content_col]

    # TF-IDF 벡터화
    vectorizer = TfidfVectorizer().fit_transform(df['combined'])
    similarity_matrix = cosine_similarity(vectorizer)

    to_drop = set()
    duplicate_pairs = []

    for i in range(similarity_matrix.shape[0]):
        for j in range(i + 1, similarity_matrix.shape[0]):
            sim = similarity_matrix[i, j]
            if sim > threshold:
                to_drop.add(j)
                duplicate_pairs.append((i, j, sim))

    if preview:
        print("🟡 유사도가 높은 항목 쌍:")
        for i, j, sim in duplicate_pairs:
            print(f"\n🔹 [{i}] 제목: {df.loc[i, title_col][:50]}")
            print(f"🔹 [{j}] 제목: {df.loc[j, title_col][:50]}")
            print(f"   ↪ 유사도: {sim:.4f}")

    # 중복 제거
    df_cleaned = df.drop(list(to_drop)).drop(columns=['combined']).reset_index(drop=True)

    return df_cleaned


In [165]:
df5 = remove_similar_entries(df4)

🟡 유사도가 높은 항목 쌍:

🔹 [0] 제목: 강아지 탈모 가려움증 원인 곰팡이성 피부병 원인 치료
🔹 [14] 제목: 강아지 탈모 가려움증 원인 곰팡이성 피부병 원인 치료
   ↪ 유사도: 1.0000

🔹 [0] 제목: 강아지 탈모 가려움증 원인 곰팡이성 피부병 원인 치료
🔹 [38] 제목: 강아지 탈모 가려움증 원인 곰팡이성 피부병 원인 치료
   ↪ 유사도: 1.0000

🔹 [0] 제목: 강아지 탈모 가려움증 원인 곰팡이성 피부병 원인 치료
🔹 [62] 제목: 강아지 탈모 가려움증 원인 곰팡이성 피부병 원인 치료
   ↪ 유사도: 1.0000

🔹 [0] 제목: 강아지 탈모 가려움증 원인 곰팡이성 피부병 원인 치료
🔹 [86] 제목: 강아지 탈모 가려움증 원인 곰팡이성 피부병 원인 치료
   ↪ 유사도: 1.0000

🔹 [0] 제목: 강아지 탈모 가려움증 원인 곰팡이성 피부병 원인 치료
🔹 [110] 제목: 강아지 탈모 가려움증 원인 곰팡이성 피부병 원인 치료
   ↪ 유사도: 1.0000

🔹 [0] 제목: 강아지 탈모 가려움증 원인 곰팡이성 피부병 원인 치료
🔹 [134] 제목: 강아지 탈모 가려움증 원인 곰팡이성 피부병 원인 치료
   ↪ 유사도: 1.0000

🔹 [0] 제목: 강아지 탈모 가려움증 원인 곰팡이성 피부병 원인 치료
🔹 [158] 제목: 강아지 탈모 가려움증 원인 곰팡이성 피부병 원인 치료
   ↪ 유사도: 1.0000

🔹 [0] 제목: 강아지 탈모 가려움증 원인 곰팡이성 피부병 원인 치료
🔹 [182] 제목: 강아지 탈모 가려움증 원인 곰팡이성 피부병 원인 치료
   ↪ 유사도: 1.0000

🔹 [0] 제목: 강아지 탈모 가려움증 원인 곰팡이성 피부병 원인 치료
🔹 [206] 제목: 강아지 탈모 가려움증 원인 곰팡이성 피부병 원인 치료
   ↪ 유사도: 1.0000

🔹 [0] 제목: 강아지 탈모 가려움증 원인 곰팡이성 피부병 원인 치료
🔹 [230] 제목: 강아지 탈모 가려움증 원인 곰팡이성 피부병 원인 치료
   ↪ 

In [166]:
len(df4), len(df5)

(4238, 2573)

In [169]:
df6 = remove_similar_entries(df5)

🟡 유사도가 높은 항목 쌍:


---

### 확인 및 저장

In [173]:
# df5.columns
# df5.isnull().sum()

title      0
content    0
url        0
image      0
dtype: int64

In [174]:
# 결과 저장 (선택)
df5.to_csv('./merged_rag_data.csv', index=False)
